# Темы семинара: отбор признаков и приложения

- Фильтрационные методы
- Оберточные методы
- Встроенные методы
- Нелинейные классификаторы
- Сохранение модели

In [4]:
import numpy as np
import pandas as pd

In [6]:
POKEMON = "./Pokemon.csv"

In [8]:
data = pd.read_csv(POKEMON)

In [10]:
data.head(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [16]:
data.isnull().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [17]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [19]:
display(data.isnull().sum())

#             0
Name          0
Type 1        0
Type 2        0
Total         0
HP            0
Attack        0
Defense       0
Sp. Atk       0
Sp. Def       0
Speed         0
Generation    0
Legendary     0
dtype: int64

In [20]:
data.drop(columns=['#'], inplace=True)

In [22]:
data.drop(columns=['Name'], inplace=True)

In [23]:
data

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Grass,Poison,525,80,82,83,100,100,80,1,False
3,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Fire,No 2nd type,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [24]:
X = data.drop(columns='Legendary')
y = data['Legendary'].astype('int')

In [25]:
y.value_counts()

Legendary
0    735
1     65
Name: count, dtype: int64

In [26]:
y.value_counts(normalize=True)

Legendary
0    0.91875
1    0.08125
Name: proportion, dtype: float64

# Make some default pipeline

In [30]:
# !pip install category_encoders

In [31]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [33]:
data.head(2)

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Grass,Poison,405,60,62,63,80,80,60,1,False


In [34]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [37]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

In [38]:
cv_res1

{'fit_time': array([0.02703691, 0.02403641, 0.02803779, 0.02903676, 0.02254653]),
 'score_time': array([0.00851893, 0.00851893, 0.00951862, 0.00951862, 0.00901103]),
 'test_score': array([0.5       , 0.72727273, 0.47619048, 0.38095238, 0.64864865]),
 'train_score': array([0.71287129, 0.56097561, 0.6744186 , 0.7311828 , 0.72727273])}

In [39]:
cv_res1['test_score'].mean()

0.5466128466128466

# Make pipeline more complicated

In [43]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [44]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

cv_res2

{'fit_time': array([0.0604372 , 0.0780561 , 0.07150149, 0.06344104, 0.08255267]),
 'score_time': array([0.01457334, 0.01255322, 0.0131073 , 0.01417017, 0.01356387]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [45]:
cv_res2['test_score'].mean()

0.5934281842818427

# Introduce feature selectors

In [47]:
data_tr = pipe_dif[:-1]

In [49]:
data_tr

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler())])

In [50]:
X.shape

(800, 10)

In [51]:
X_tr = data_tr.fit_transform(X, y)
print(f'data shape after transformation is {X_tr.shape}')

data shape after transformation is (800, 1001)


1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

 - статистика хи-квадрат
 - метрика mutual information

In [54]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [55]:
k_best = 30

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func=mutual_info_classif, k=k_best)),
    ('model_', SVC(kernel='linear'))]
)

In [56]:
cv_res = cross_validate(pipe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res

{'fit_time': array([3.24951243, 3.26229334, 3.18310833, 3.26355147, 3.26919985]),
 'score_time': array([0.01000428, 0.00999999, 0.00954485, 0.0090003 , 0.00999951]),
 'test_score': array([0.375     , 0.72727273, 0.61538462, 0.4       , 0.26666667]),
 'train_score': array([0.78      , 0.60674157, 0.7755102 , 0.75510204, 0.69565217])}

In [57]:
# k best нужно подбирать

cv_res['test_score'].mean()

0.4768648018648019

## Жадный метод отбора

In [59]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [60]:
rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [61]:
X_tr.shape

(800, 1001)

In [62]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

(800, 30)

array([[-0.98555744, -0.43377084, -0.966347  , ..., -0.85654781,
        -0.41626469, -0.72668962],
       [-0.48479877, -0.34070105, -0.966347  , ..., -0.75798867,
        -0.10954899, -0.70651232],
       [ 0.42451538, -0.18186195, -0.966347  , ..., -0.52902354,
         0.67405053, -0.6731154 ],
       ...,
       [-0.16049792,  2.60136793, -0.966347  , ...,  2.88884319,
         4.39362648,  4.87819432],
       [-0.16049792,  3.02541161, -0.966347  , ...,  5.4302028 ,
         5.8291638 ,  4.87819432],
       [ 1.36562373,  0.35227348, -0.966347  , ...,  1.70824175,
         1.1636675 ,  1.93926564]])

In [63]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter=1000),
                      n_features_to_select=30,
                      step=30
                     )),
    ('model_', SVC(kernel='linear'))])

In [64]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

{'fit_time': array([1.36097002, 1.55796027, 1.23204112, 1.53109789, 1.39286828]),
 'score_time': array([0.00983429, 0.0098772 , 0.01124454, 0.01000428, 0.00999999]),
 'test_score': array([0.42105263, 0.88888889, 0.68965517, 0.58333333, 0.66666667]),
 'train_score': array([0.88888889, 0.8172043 , 0.94339623, 0.875     , 0.94230769])}

In [65]:
cv_res3['test_score'].mean()

0.6499193385763259

## С помощью L1 регуляризации

In [67]:
from sklearn.feature_selection import SelectFromModel

In [2]:
sel = SelectFromModel(LogisticRegression(penalty='l1', 
                                         max_iter=1000, 
                                         solver='liblinear'), threshold=1e-5)

NameError: name 'SelectFromModel' is not defined

In [69]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

(800, 50)

array([[-1.39963712, -1.20562657, -0.44705251, ..., -0.52135831,
        -0.72668962, -0.81966779],
       [-1.39963712, -1.20562657, -0.37458929, ..., -0.18003271,
        -0.70651232, -0.81698899],
       [-1.39963712, -1.20562657, -0.24576578, ...,  0.73602435,
        -0.6731154 , -0.81341726],
       ...,
       [ 1.61195431, -1.20562657,  0.66347512, ...,  2.1076934 ,
         4.87819432,  4.18343684],
       [ 1.61195431, -1.20562657,  1.03195307, ...,  2.51608978,
         4.87819432,  4.18343684],
       [ 1.61195431, -1.20562657, -0.13354138, ...,  0.19750623,
         1.93926564,  2.6404483 ]])

In [70]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'),
                                  threshold=1e-5)),
    ('model_', SVC(kernel='linear'))])

In [71]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res4

{'fit_time': array([0.09036636, 0.09952211, 0.09231544, 0.08625722, 0.09316111]),
 'score_time': array([0.01300621, 0.0090003 , 0.00999999, 0.00899982, 0.01000023]),
 'test_score': array([0.375     , 0.88      , 0.68965517, 0.60869565, 0.68421053]),
 'train_score': array([0.92307692, 0.875     , 0.95238095, 0.93877551, 0.92929293])}

In [72]:
cv_res4['test_score'].mean()

0.6475122701806991

# Нелинейные классификаторы

- SVM с ядром
- Наивный байесовский классификатор
- Метод k ближайших соседей

In [74]:
pipe_lasso2 =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'),
                                  threshold=1e-5)),
    ('model_', SVC(kernel='rbf'))])

In [75]:
cv_res5 = cross_validate(pipe_lasso2, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([0.08970165, 0.10299635, 0.09456587, 0.08999515, 0.0978229 ]),
 'score_time': array([0.01100349, 0.0120039 , 0.0120039 , 0.0120039 , 0.01100326]),
 'test_score': array([0.35294118, 0.7       , 0.52173913, 0.42105263, 0.64864865]),
 'train_score': array([0.88888889, 0.83146067, 0.85106383, 0.90526316, 0.92929293])}

In [76]:
cv_res5['test_score'].mean()

0.5288763174265934

Обучите наивный байесовский классификатор и метод k ближайших соседей вместо SVM в пайплайне выше.

In [78]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [79]:
# your code here
pipe_knn =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'),
                                  threshold=1e-5)),
    ('model_', KNeighborsClassifier())])

In [80]:
pipe_knn

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler()),
                ('selector_',
                 SelectFromModel(estimator=LogisticRegression(max_iter=1000,
                                                              penalty='l1',
                                                              solver='liblinear'),
                                 threshold=1e-05)),
                ('model_', KNeighborsClassifier())])

In [81]:
cv_res5 = cross_validate(pipe_knn, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([0.08561969, 0.09500337, 0.09080839, 0.08698273, 0.09110141]),
 'score_time': array([0.30094075, 0.02001905, 0.01999998, 0.02201724, 0.02155375]),
 'test_score': array([0.47058824, 0.72727273, 0.38095238, 0.33333333, 0.58064516]),
 'train_score': array([0.76595745, 0.77894737, 0.73333333, 0.77777778, 0.78787879])}

In [82]:
cv_res5['test_score'].mean() #50

0.49855836762857636

С помощью GridSearch подберите гиперпараметр KNN (число соседей) внутри пайплайна.

In [84]:
from sklearn.model_selection import GridSearchCV

In [85]:
pipe_knn =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'),
                                  threshold=1e-5)),
    ('gridsearch_', GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors' : range(1,51)}))])

In [86]:
pipe_knn =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'),
                                  threshold=1e-5)),
    ('gridsearch_', GridSearchCV(estimator=SVC(kernel='rbf'),
             param_grid={'C' : range(1, 5)}))])

In [87]:
cv_res5 = cross_validate(pipe_knn, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([0.20683885, 0.21718884, 0.21190047, 0.1994381 , 0.20799637]),
 'score_time': array([0.01200342, 0.01153302, 0.01153064, 0.01100349, 0.01100326]),
 'test_score': array([0.375     , 0.7       , 0.60869565, 0.4       , 0.64864865]),
 'train_score': array([0.92156863, 0.83146067, 0.88421053, 0.92783505, 0.92929293])}

In [88]:
cv_res5['test_score'].mean()

0.5464688601645123

# Сохранение и загрузка модели

In [90]:
import pickle

Обучим лучшую модель на всех данных и сохраним её в файл.

In [92]:
model = pipe_knn

In [93]:
filename = 'best_model.pickle'
pickle.dump(model, open(filename, 'wb'))

Загрузка модели из файла

In [96]:
loaded_model = pickle.load(open(filename, 'rb'))

Второй вариант

In [98]:
import joblib

filename = 'best_model.joblib'
joblib.dump(model, filename)

loaded_model = joblib.load(filename)

Почитать подробнее про сохранение модели в файл и загрузку из файла тут: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/